In [2]:
#################
# LOAD PACKAGES #
#################

library(tidyverse)
library(viridis)
library(dplyr)
library(lubridate)
library(patchwork)

###################
# SET DIRECTORIES #
###################

setwd("~//OneDrive/git-repos/EMMA/")

input_dir <- "isco_metadata/LCBP_RI_sample_index/"

################
# READ IN DATA #
################

# read in the streamwater ISCO data
InputData <- read.csv(file.path(input_dir, "RI23-IC-ICP-isotope-toc-joined.csv")) 

InputData <- InputData %>% # Will fix this, but some endmembers misslabeled
  mutate(Type = case_when(
    Type == "Snow lysimeter" ~ "Snowmelt lysimeter",
    TRUE ~ Type
  ))

# Convert missing values to NA
InputData[InputData == ""] <- NA

# Combine 'Date' and 'Time' columns into 'Datetime' 
InputData$datetime <- mdy_hm(paste(InputData$Date, InputData$Time)) 

# Convert 'datetime' in InputDataQ to proper datetime format
InputData$datetime <- as.POSIXct(InputData$datetime, format="%Y-%m-%dT%H:%M:%SZ")

###############################################################################
################## PLOT ENDMEMBER VARIABILITY IN TIME #########################
###############################################################################

# Step 1: Define tracer list and sample types to keep
tracers <- c("Na_mg_L", "Si_mg_L", "Ca_mg_L", "Mg_mg_L", 
             "K_mg_L", "Cl_mg_L", "dD", "d18O", "TOC.mg_L")

endmember_types <- c(
  #"Soil water lysimeter dry",
  #"Soil water lysimeter wet",
  #"Snowmelt lysimeter",  # Already renamed from "Snow lysimeter"
  #"Snow",
  #"Rain",
  #"Precip",
  "Groundwater",
  #"Ravindra deep groundwater",
  "Baseflow",
  "Pre-event"
)

# Step 2: Filter and pivot data into long format
InputData_long <- InputData %>%
  filter(Type %in% endmember_types) %>%
  select(Site, datetime, Type, all_of(tracers)) %>%
  pivot_longer(
    cols = all_of(tracers),
    names_to = "tracer",
    values_to = "value"
  ) %>%
  mutate(
    value = as.numeric(value),
    Type = trimws(Type)
  )

# Step 3: Define colors and shapes (same as earlier)
custom_colors <- c(
  "Soil water lysimeter dry" = "black",
  "Soil water lysimeter wet" = "#4F4F4F",
  "Groundwater" = "black",
  "Precip" = "grey",
  "Rain" = "grey",
  "Snowmelt lysimeter" = "black",
  "Baseflow" = "black",
  "Pre-event" = "black",
  "Ravindra deep groundwater" = 'black',
  "Snow" = "black"
)

custom_shapes <- c(
  "Soil water lysimeter dry" = 23,
  "Soil water lysimeter wet" = 17,
  "Groundwater" = 15,
  "Precip" = 17,
  "Rain" = 17,
  "Snowmelt lysimeter" = 19,
  "Baseflow" = 18,
  "Pre-event" = 8,
  "Ravindra deep groundwater" = 19,
  "Snow" = 8
)

# Step 4: Plotting function per site

options(repr.plot.width = 10, repr.plot.height = 10) 

plot_tracer_timeseries <- function(site_name) {
  InputData_long %>%
    filter(Site == site_name) %>%
    ggplot(aes(x = datetime, y = value, color = Type, shape = Type)) +
    geom_point(size = 5, alpha = 1.0) +
    scale_color_manual(values = custom_colors) +
    scale_shape_manual(values = custom_shapes) +
    facet_wrap(~ tracer, scales = "free_y", ncol = 2, strip.position = "right") +
    labs(
      title = paste0(site_name, ": Groundwater tracer timeseries"),
      x = "Date",
      y = "Conc [mg/L] or ‰"
    ) +
    theme_minimal(base_size = 20) +
theme(
  panel.grid = element_blank(),                 # Removes all grid lines
  strip.text = element_text(face = "bold"),
  legend.position = "bottom",
  panel.border = element_rect(color = "black", fill = NA, linewidth = 0.8),  # Boxes around facets
  strip.background = element_rect(fill = "#f5f5f5", color = NA)              # Optional: soft background behind facet labels
)
}

# Step 5: Create plots for each site
plot_hungerford <- plot_tracer_timeseries("Hungerford")
plot_wade <- plot_tracer_timeseries("Wade")

# Optionally: Display them one after the other
plot_hungerford

SyntaxError: invalid syntax (603032210.py, line 26)